In [11]:
import os
from pprint import pprint
import requests
import pandas as pd
import numpy as np
import time
import csv
import json
import datetime

# README: How this script works and is used.
All out from the functions defined, only "getIssueTablebyProject" is being used for issue crawling.
In the next cell, making the function call in a for loop for all the projects, there's a piece of code that will create the empty csv files matching the required format for the function that is going to be used.

Once this is used, it must be commented out and the function call must be made to fill all the csv files.


In [20]:
personal_token = ""
token = os.getenv('GITHUB_TOKEN', personal_token)
headers = {'Authorization': f'token {token}'}

desired_width=640
pd.set_option('display.width', desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option('display.max_columns',25)

# What is inside the txt file???
'''
with open('./ProphetOutcomes/ts.txt', 'r') as txtfile:
    tslist = [x.strip('\n').split()[2].split('/')[1] for x in txtfile.readlines()]
'''

# Will take requestCounter as a parameter and return it so that the function keeps count of the
# amount of requests made.
def getIssueTablebyProject(projectfullname, updateissuetablename, requestCounter): #, updatelabeltablename):

    theIssueQuery = f"https://api.github.com/repos/{projectfullname}/issues"
    theProjectQuery = f"https://api.github.com/repos/{projectfullname}"
    p_search = requests.get(theProjectQuery, headers=headers)
    requestCounter += 1 # Each time a request is made we monitorize it.

    project_info = p_search.json()
    project_id = project_info['id']
    issue_feature_list = ['project_id', 'issue_count', 'unique_user_count', 'type_user_count', 'type_organization_count', 'type_bot_count', 'open_count', 'closed_count', 'locked_count', 'average_active_time', 'milestone_related_count', 'average_comments']
    label_feature_list = ['project_id', 'issue_id', 'label_id', 'name', 'color', 'default', 'description']
    #issueheader = ','.join(issue_feature_list)
    #labelheader = ','.join(label_feature_list)
    #with open(updateissuetablename, 'a', encoding='utf-8') as csvfile:
    #    writer = csv.writer(csvfile, delimiter=',')
    #    writer.writerow(issueheader)
    #with open(updatelabeltablename, 'a', encoding='utf-8') as csvfile:
    #    writer = csv.writer(csvfile, delimiter=',')
    #    writer.writerow(labelheader)
    params = {'per_page': 100, 'state': 'all'}
    #theOpenIssueList = []
    page = 1
    projectissuedataitems = []
    thereturnvalues = {}
    labeldataitems = []
    while 1 == 1:
        params['page'] = page
        print(projectfullname+' '+'page '+str(page))
        theResult = requests.get(theIssueQuery, headers=headers, params=params)
        requestCounter += 1 # Each time a request is made we monitorize it.

        theItemListPerPage = theResult.json()
        if len(theItemListPerPage) == 0:
            break
        else:
            for item in theItemListPerPage:
                issueitem = {}
                issueitem['project_id'] = project_id
                issueitem['issue_id'] = item['id']
                issueitem['number'] = item['number']
                issueitem['user_login'] = item['user']['login']
                issueitem['user_type'] = item['user']['type']
                issueitem['state'] = item['state']
                issueitem['locked'] = item['locked']
                issueitem['created_at'] = (pd.to_datetime(item['created_at'], utc=True)- pd.Timestamp("1970-01-01").tz_localize('UTC'))// pd.Timedelta('1s')
                if item['updated_at'] == None:
                    issueitem['updated_at'] = np.NaN
                else:
                    issueitem['updated_at'] = (pd.to_datetime(item['updated_at'], utc=True)- pd.Timestamp("1970-01-01").tz_localize('UTC'))// pd.Timedelta('1s')
                if item['closed_at'] == None:
                    issueitem['closed_at'] = np.NaN
                else:
                    issueitem['closed_at'] = (pd.to_datetime(item['closed_at'], utc=True)- pd.Timestamp("1970-01-01").tz_localize('UTC'))// pd.Timedelta('1s')
                if issueitem['state'] == 'closed':
                    issueitem['active_time'] = issueitem['closed_at'] - issueitem['created_at']
                else:
                    if not issueitem['updated_at']:
                        issueitem['active_time'] = 0
                    else:
                        issueitem['active_time'] = issueitem['updated_at'] - issueitem['created_at']
                #issueitem['title'] = item['title']
                #issueitem['body'] = item['body']
                if item['milestone'] == None:
                    issueitem['milestone_url'] = False
                else:
                    issueitem['milestone_url'] = True
                issueitem['comments'] = item['comments']
                if 'pull_request' in list(item.keys()):
                    issueitem['pull_request'] = True
                else:
                    issueitem['pull_request'] = False
                #with open(updateissuetablename, 'a', encoding='utf-8') as csvfile:
                #    writer = csv.writer(csvfile, delimiter=',')
                #    writer.writerow([issueitem[x] for x in issue_feature_list])
                projectissuedataitems.append(issueitem)
                ############# get labels ################
                #if item['labels'] == None:
                #    pass
                #else:
                #    for label in item['labels']:
                #        labelitem = {}
                #        labelitem['project_id'] = project_id
                #        labelitem['issue_id'] = item['id']
                #        labelitem['label_id'] = label['id']
                #        labelitem['name'] = label['name']
                #        labelitem['color'] = label['color']
                #        labelitem['default'] = label['default']
                #        if label['description'] == None:
                #            labelitem['description'] = np.NaN
                #        else:
                #            labelitem['description'] = label['description']
                #        labeldataitems.append(labelitem)
                #        with open(updatelabeltablename, 'a', encoding='utf-8') as csvfile:
                #            writer = csv.writer(csvfile, delimiter=',')
                #            writer.writerow([labelitem[x] for x in label_feature_list])
                ###########################################
            #theOpenIssueList.extend(theItemListPerPage)
            page = page + 1
        #issue_feature_list = ['project_id', 'issue_count', 'unique_user_count', 'type_user_count', 'type_organization_count', 'type_bot_count',
        #                      'open_count', 'closed_count', 'locked_count', 'average_active_time',
        #                      'milestone_related_count', 'average_comments', 'pull_request_count']
    thereturnvalues['project_id'] = project_id
    thereturnvalues['issue_count'] = len(projectissuedataitems)
    thereturnvalues['unique_user_count'] = len(list(set([x['user_login'] for x in projectissuedataitems])))
    thereturnvalues['type_user_count'] = len([x['user_type'] for x in projectissuedataitems if x['user_type']=='User'])
    thereturnvalues['type_organization_count'] = len([x['user_type'] for x in projectissuedataitems if x['user_type']=='Organization'])
    thereturnvalues['type_bot_count'] = len([x['user_type'] for x in projectissuedataitems if x['user_type']=='Bot'])
    thereturnvalues['open_count'] = len([x['state'] for x in projectissuedataitems if x['state']=='open'])
    thereturnvalues['closed_count'] = len([x['state'] for x in projectissuedataitems if x['state']=='closed'])
    thereturnvalues['locked_count'] = sum([x['locked'] for x in projectissuedataitems])
    try:
        thereturnvalues['average_active_time'] = sum([x['active_time'] for x in projectissuedataitems])/len(projectissuedataitems)
        thereturnvalues['average_closed_time'] = sum([x['active_time'] for x in projectissuedataitems if x['state']=='closed'])/len(projectissuedataitems)
    except ZeroDivisionError:
        thereturnvalues['average_active_time'] = np.nan
        thereturnvalues['average_closed_time'] = np.nan
    thereturnvalues['milestone_related_count'] = sum([x['milestone_url'] for x in projectissuedataitems])
    try:
        thereturnvalues['average_comments'] = sum([x['comments'] for x in projectissuedataitems])/len(projectissuedataitems)
    except ZeroDivisionError:
        thereturnvalues['average_comments'] = np.nan

    thereturnvalues['pull_request_count'] = sum([x['pull_request'] for x in projectissuedataitems])
    features = list(thereturnvalues.keys())
    currentdf = pd.read_csv(updateissuetablename, index_col='project_id')
    existingprojects = currentdf.index.values.tolist()
    #existingprojects = []
    if project_id in existingprojects:
        currentdf.loc[project_id] = [thereturnvalues[x] for x in features][1:]
        currentdf.reset_index(inplace=True)
        currentdf.to_csv(updateissuetablename, encoding='utf-8', index=False)
    else:
        with open(updateissuetablename, 'a', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([thereturnvalues[x] for x in features])

    return requestCounter

In [24]:
# pandora.csv file is taken from the software Pandora, Dario helped me to understand how the application
# works and I took the list of apache projects that at some point used sonarQube.
repos_github = pd.read_csv("cleanRepos070223.csv")

# Converts the column array of projects' full names into list.
github_fullNames = list(repos_github['full_name'])

# Issue features that will make the set of columns of our newly created csv files.
issue_feature_list = ['project_id', 'issue_count', 'unique_user_count', 'type_user_count', 'type_organization_count', 'type_bot_count', 'open_count', 'closed_count', 'locked_count', 'average_active_time', 'average_active_time','milestone_related_count', 'average_comments', 'pull_request_count']

# NOTE: Creates the header files for all the projects. Now, when there's already something in the files the second for loop will throw an error,
# So, before the script is run, the files need to be recreated and then run the second for loop.
'''
for i in range(len(github_fullNames)):

    with open(f"/home/mikel/Desktop/project1/{github_fullNames[i]}.csv", "w") as updateissuetablename:
        writer = csv.writer(updateissuetablename, delimiter=',')
        writer.writerow(issue_feature_list)
'''

requestCounter = 0
for i in range(len(github_fullNames)):
    updateissuetablename = f"/home/mikel/Desktop/project1/{github_fullNames[i]}.csv"

    requestCounter += getIssueTablebyProject(github_fullNames[i], updateissuetablename, requestCounter)
    print(f"Resquest Nº: {requestCounter}, project name/number: {github_fullNames[i]}/{i}")


apache/tapestry3 page 1
apache/tapestry3 page 2
Resquest Nº: 3, project name/number: apache/tapestry3/0
apache/apr-iconv page 1
apache/apr-iconv page 2
Resquest Nº: 9, project name/number: apache/apr-iconv/1
apache/tapestry4 page 1
apache/tapestry4 page 2
Resquest Nº: 21, project name/number: apache/tapestry4/2
apache/sling-old-svn-mirror page 1
apache/sling-old-svn-mirror page 2
apache/sling-old-svn-mirror page 3
apache/sling-old-svn-mirror page 4
Resquest Nº: 47, project name/number: apache/sling-old-svn-mirror/3
apache/xalan-j page 1
apache/xalan-j page 2
Resquest Nº: 97, project name/number: apache/xalan-j/4
apache/apr page 1
apache/apr page 2
Resquest Nº: 197, project name/number: apache/apr/5
apache/stdcxx page 1
Resquest Nº: 396, project name/number: apache/stdcxx/6
apache/zookeeper page 1
apache/zookeeper page 2
apache/zookeeper page 3
apache/zookeeper page 4
apache/zookeeper page 5
apache/zookeeper page 6
apache/zookeeper page 7
apache/zookeeper page 8
apache/zookeeper page 9
